In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import math as m
import numpy.linalg as la
import scipy.linalg as sla
import csv
import matplotlib 
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20) 

#Boeing Insitu ScanEagle Numbers (ON EARTH)
g = 9.81 #m/s^2
w1 = (18+14)/2 #avg empty structure weight in kg
w0 = 22 #kg (max takeoff weight)
Vmax = 41.2 #m/s max horisontal speed
Vcruise = 25 #cruise speed in m/s
b = 3.11 #m (wingspan)
c = 3.11/13 #m (chord length using pixel counting)
Ceiling = 5944 #m
ServiceC = 4572 #m
Sref = b*c

#Boeing Insitu ScanEagle Numbers (ON TITAN)
gt = 1.352 #m/s^2 (t = Titan)
w1t = (w1)*gt #avg empty structure weight in kg
w0t = (w0)*gt #max takeoff weight in kg



In [3]:
#read csv
temp = []
altitudes = []
with open("TitanTempAlt.csv",'r') as csvfile:
    plots = csv.reader(csvfile, dialect='excel', delimiter=',')
    for row in plots:
        try:
            altitudes.append(float(row[0]))
            temp.append(float(row[1]))
        except:
            continue
            
def linear_interpolator(para1, para2, alt1, alt2, alt):
    alt_ratio = (alt - alt1) / (alt2 - alt1)
    return ((para2 - para1) * alt_ratio) + para1

def atms_conditions(alt):
    #get alt in km to work with csv data which is in km
    #alt = alt/1000
    if alt > altitudes[-1] or alt <= 0:
        print("sorry this altitude is not in range of our function")
        return
    
    P0 = 146700 #Pa
    R  = 296.8  #J/kg * K
    rho0 = P0 / (temp[0] * R)
    g = 1.352 #m/s^2; just googled value for titan
    #exponential relation for density
    #print(alt)
    R_titan = 2575 #km
    P1 = P0 * m.exp(-alt/(R_titan))
    #P1 = P0 * e^(-alt/Radius)
    #print(P1)
    #print(rho0)
    iterator = 0 #describes state state of while loop (ie what altitude and temp to be at)
    next_temp = temp[0]
    #lists akin to vectors, allow for dynamic storage of values
    while altitudes[iterator] < alt:
        #get the temp and altitude of the next state of function
        next_temp = temp[iterator + 1]
        next_altitude = altitudes[iterator + 1]
        iterator += 1
    
    T_ceil = temp[iterator]
    T_floor = temp[iterator - 1]
    alt_ceil = altitudes[iterator]
    alt_floor = altitudes[iterator - 1]
    T1 = linear_interpolator(T_floor, T_ceil, alt_floor, alt_ceil, alt)
    #print(T_floor, T_ceil, T1)
    
    rho1 = P1 / (R * T1)
    
    #print(rho1)
    
    outputs = np.array([alt*1000, P1, T1, rho1])
    #print(rho1)
    return rho1

print(atms_conditions(0.1))

5.28946543819868


In [4]:
def LiftCoeff(W, vlist, Sref, rho): 
    CL = W/(0.5*rho*Sref*vlist**2)
    return CL

In [5]:
# INITIAL VALUES
cd0 = 0.0072
PA = 1200 # wats
PA_list = np.linspace(1200, 1200, 1000)

# Run our atomosperic calculations for all altitudes
alt_list = np.arange(1, 141, 1)
rho_list = np.empty(140)
rho_list = [atms_conditions(alt) for alt in alt_list]
rho_list = np.array(result_list)
rho_list[0] = 0

# Calculate CL for all the values of rho
vlist = np.linspace(0, Vmax, 1000)
CL_list = [LiftCoeff(w0t, vlist, Sref, rho) for rho in rho_list]

# Calculate CD for all the values of CL
CD_list = [cd0 + CL**2/(np.pi*0.8*13.) for CL in CL_list]

CL_list = np.array(CL_list)
CD_list = np.array(CD_list)

# Calculate Thrsut Required
TR_list = [w0t/(CL_list[i,:]/CD_list[i,:]) for i in range(np.shape(CL_list)[0])]
TR_list = np.array(TR_list)

# Calculate Power Required
PR_list = [TR * vlist for TR in TR_list]
PR_list = np.array(PR_list)

# Calculate Rate of Climb
ROC_list = [(PA - PR) / w0t for PR in PR_list]
ROC_list = np.array(ROC_list)
ROC_list[:, 0] = 0 #Remove the point [0, infinity]                                        

# Calculate max ROC
ROC_MAX_list = []
for i in range(np.shape(ROC_list)[0]):
    ROC_MAX_list.append(np.max(ROC_list[i, :]))
    
# Determine which altitudes exceed our avaliable power
P_EQUAL_indexes_v = []
P_EQUAL_indexes_a = []
for i in range(np.shape(ROC_list)[0]):
    for j in range(np.shape(ROC_list)[1]):
        if (PR_list[i, j] >= PA):
            P_EQUAL_indexes_v.append(j)
            P_EQUAL_indexes_a.append(i)
            break;
            

P_EQUAL_indexes_v = np.array(P_EQUAL_indexes_v)
P_EQUAL_indexes_a = np.array(P_EQUAL_indexes_a)

print(P_EQUAL_indexes_a)

#V_EQUAL_velocities = np.sort(vlist[P_EQUAL_indexes])



########## ALL PLOTTING FROM THIS POINT ONWARDS

plt.yscale('log')
plt.xscale('log')
plt.title("Lift Coefficient vs Velocity")
plt.xlabel("Velocity (m/s)")
plt.ylabel("Lift Coefficient")

i = 500
for CL in CL_list:
    plt.plot(vlist, CL, label="Alt={}".format(i))
    i+=500
    
plt.title("CD vs Velocity")
#plt.legend()
plt.show()

i = 500
for CD in CD_list:
    plt.plot(vlist, CD, label="Alt={}".format(i))
    i+=500
plt.yscale('log')
plt.xscale('log')
plt.title("CD vs Velocity")
#plt.legend()
plt.show()

for i in range(np.shape(CL_list)[0]):
    plt.plot(CL_list[i], CD_list[i], label="Alt={}".format(i))

plt.title("CD vs CL")
#plt.legend()
plt.show()

print(np.shape(vlist), np.shape(ROC_list))

for i in range(np.shape(ROC_list)[0]):
    plt.plot(vlist, ROC_list[i, :])
plt.title("Velocity vs ROC")
plt.show()

for i in range(np.shape(PR_list)[0]):
    plt.plot(vlist, PR_list[i], label = 'PR{}'.format(i))
plt.plot(vlist, PA_list, label = 'PA')
plt.title("PA and PR vs velocity")
#plt.legend()
plt.show()



NameError: name 'result_list' is not defined

In [ ]:
print(np.shape(ROC_MAX_list), np.shape(alt_list))

print(alt_list)


plt.plot(ROC_MAX_list, alt_list)
plt.title("Max ROC vs Height")
plt.show()

In [ ]:
# 61-80: Condensate Haze
# 80-140: Tholin Haze
# 17-35: CH4-N2 Clouds
# Lower Surafce

CH_upper = np.linspace(80,80,140)
CH_lower = np.linspace(61,61,140)
TH_lower = np.linspace(80, 80,140)
TH_upper = np.linspace(140,140,140)
CH4_upper = np.linspace(35,35,140)
CH4_lower = np.linspace(17,17,140)

len(ROC_MAX_list)

In [ ]:
plt.figure(figsize=(27,15))
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 20}

matplotlib.rc('font', **font)
plt.plot(ROC_MAX_list, alt_list, linewidth=10, color='cornflowerblue')
plt.plot(ROC_MAX_list, CH_upper, linewidth=2, color='BROWN', label = "Condensate Haze Upper Bound and Tholin Haze Lower Bound")
plt.plot(ROC_MAX_list, CH_lower, linewidth=2, color='crimson', label ="Condensate Haze Lower Bround")
plt.plot(ROC_MAX_list, TH_upper, linewidth=2, color='darkorange', label = 'Tholin Haze Upper Bound')
plt.plot(ROC_MAX_list, CH4_upper, linewidth=2, color='magenta', label = 'CH4-N2 Clouds Upper Bound')
plt.plot(ROC_MAX_list, CH4_lower, linewidth=2, color='darkmagenta', label = 'CH4-N2 Clouds Lower Bound')
#plt.plot(CH_upper)
plt.title("Altitude vs Rate of Climb")
plt.xlabel("Rate of Climb (km/s)")
plt.ylabel("Altitude (km)")
#plt.legend(bbox_to_anchor=(-0.1,0), loc="lower right",  prop={'size': 50})
plt.savefig('img')
plt.show()


In [ ]:
plt.figure(figsize=(27,15))
plt.xscale('log')
plt.yscale('log')
plt.title("Coefficient of Drag vs Velocity at 5km")
plt.xlabel("Velocity (m/s)")
plt.ylabel("CD")
plt.plot(vlist, CD_list[6], linewidth=10, color="olivedrab")
plt.savefig("CD5k")

In [ ]:
plt.figure(figsize=(27,15))
plt.xscale('log')
plt.yscale('log')
plt.title("Coefficient of Lift vs Velocity at 5km")
plt.xlabel("Velocity (m/s)")
plt.ylabel("CL")
plt.plot(vlist, CL_list[6], linewidth=10, color="orangered")
plt.savefig("CL5k")

In [ ]:
plt.figure(figsize=(27,15))
plt.xscale('log')
plt.yscale('log')
plt.title("Coefficient of Drag vs Coefficient of Lift at 5km")
plt.xlabel("CD")
plt.ylabel("CL")
plt.plot(CD_list[6], CL_list[6], linewidth=10, color="midnightblue")
plt.savefig("CDCL")

In [ ]:
plt.figure(figsize=(27,15))
plt.title("Rate of Climb vs Velocity at 5km")
plt.xlabel("Velocity (m/s)")
plt.ylabel("Rate of Climb")
plt.plot(vlist[20:], ROC_list[6, 20:], linewidth=10, color="dimgrey")
plt.savefig("CDv")

In [6]:
#solve for required thrust to take off

def Tlo(Slo,muR,rho,v):
    cl = 0.2 #0.1637 for max power req
    cd = 0.0072 + ((cl**2)/(m.pi*0.8*13))
    D = 0.5*rho*(v**2)*Sref*cd
    L = 0.5*rho*(v**2)*Sref*cl
    Tlo = ((1.44*w0t**2)/(Slo*gt*rho*Sref*cl)) + (D + muR*(w0t-L))
    return Tlo

print(Tlo(30,0.09,atms_conditions(0.1),Vmax))

def PRtr(tr,v):
    Pr = tr*v
    return Pr

print(PRtr(Tlo(30,0.09,atms_conditions(0.1),Vmax),Vmax))

#for 30 m runway, using almost max power the angle of attack is 
#at maxish possible cl where we need only 10.6 N of thrust i.e 436 watts of power, alpha is -1.5256




def Tlo(Slo,muR,rho,v):
    cl = 0.35 
    cd = 0.0072 + ((cl**2)/(m.pi*0.8*13))
    D = 0.5*rho*(v**2)*Sref*cd
    L = 0.5*rho*(v**2)*Sref*cl
    Tlo = ((1.44*w0t**2)/(Slo*gt*rho*Sref*cl)) + (D + muR*(w0t-L))
    return Tlo

def PRtr(tr,v):
    Pr = tr*v
    return Pr

print(Tlo(10,0.09,atms_conditions(0.1),Vmax))
print(PRtr(Tlo(10,0.09,atms_conditions(0.1),Vmax),Vmax))

10.60003003465663
436.72123742785317
2.447777847881767
100.8484473327288


In [7]:
#for alpha of 0.6489 deg using half v
def Tlo(Slo,muR,rho):
    cl = 1.627948514
    cd = 0.0072 + ((cl**2)/(m.pi*0.8*13))
    v = 1.2*0.7*math.sqrt((2*w0t)/(rho*Sref*cl))
    D = 0.5*rho*(v**2)*Sref*cd
    L = 0.5*rho*(v**2)*Sref*cl
    Tlo = ((1.44*w0t**2)/(Slo*gt*rho*Sref*cl)) + (D + muR*(w0t-L))
    return Tlo

print(Tlo(30,0.09,atms_conditions(0.1)))
def PRtr(tr,rho):
    cl = 1.627948514
    v = 1.2*0.7*math.sqrt((2*w0t)/(rho*Sref*cl))
    Pr = tr*v
    return Pr

print(PRtr(Tlo(30,0.09,atms_conditions(0.1)),atms_conditions(0.1)))

NameError: name 'math' is not defined

In [ ]:
L = .5*atms_conditions(0.1)*(25**2)*Sref* 